Import SolidPython and viewscad:

In [1]:
from solid import *
import viewscad

r = viewscad.Renderer()

Let's create an object:

In [2]:
c = cylinder(r=5, h=2)
c -= minkowski() (translate([0, 0, 1]) (cylinder(r=3, h=1.5)), sphere(r=0.2))
c -= cylinder(r=1, h=3, center=True)
c -= translate([0, -1, 0]) (cube([6, 2, 3]))
r.render(c)

The renderer can also render raw OpenSCAD code, and save objects to .STL files:

In [ ]:
scad_code = '''
union() {
    cube([1,1,10]);
    sphere(r=2);
}
'''
r.render(scad_code, outfile='tmp.stl')

A more complex object: a Sierpinski tetrahedron:

In [ ]:
import numpy as np

points=np.array([[0,0,0], [1,0,0],[0.5,0.5*np.sqrt(3), 0], [0.5, 0.5/np.sqrt(3), np.sqrt(2./3)]])
def tetra():
    return polyhedron(points=points, faces=[[0,1,2], [0,3,1], [0,2,3], [1,3,2]])

def sierpinski(depth, size):
    if depth == 0:
        return scale(size) (tetra())
    else:
        obj = None
        for point in points:
            o = translate(size/2*point) (sierpinski(depth-1, size/2))
            if obj is None: obj = o
            else: obj += o
        
        return obj
    
r.render(sierpinski(3, 1.))

## Introspection
Let's create a tetrahedron with rounded edges:

In [ ]:
t = hull() (sum([translate(x) (sphere(r=1.0)) for x in [[0,0,0], [20, 0, 0], [0, 20, 0], [0, 0, 30]]]))
r.render(t)
         

But say we want the big face to be downwards.  We could of course do some trigonometry, but we can also just double-click on the big face, where we learn the face index, which in my case was 4 (but could be different for you).  We can then create a new object with this face rotated downwards:

In [ ]:
t2 = r.rotate_face_down(t, 4) (t)
r.render(t2)

Let's say we had a cone which we wanted to emerge from one of the sides of t2, our new object. We can do this with the `Renderer.place_on()` method:

In [ ]:
cone = cylinder(h=15., r1=4.0, r2=0.0)
r.render(cone)

In [ ]:
new_obj = r.place_on(cone, t2, align_face1=28, align_face2=25, point1=r.get_face_centroid(cone, 28), point2=r.get_face_centroid(t2, 25))
r.render(new_obj)